In [1]:
import os
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly import tools
from ipynb.fs.defs.add_features import add_features

In [2]:
def create_chart(ticker, api):
    """ This function creates charts with the following features:
    Bollinger Bands, Moving Averages, Avg Volume, Volume, Candlesticks
    
    Parameters:
    ticker(string): the stock ticker
    api(string): alpaca api to scrape historical stock data
    
    Returns a interactive chart of the stock"""
    
    # Daily OHLCV dataframe for stock
    df = pdr.get_data_tiingo(ticker, api_key=api)
    df = df.tail(1000)
    df.reset_index(inplace=True)
    df.index = df['date']
    
    # Add additional features
    df = add_features(df)
    
    # Add candle sticks for OHLC prices
    trace0 = go.Candlestick(x=df.index,
                            open=df['open'], high=df['high'],
                            low=df['low'], close=df['close'], 
                            name='Stock Price', yaxis='y')
    
    # Add bollinger bands
    trace1 = go.Scatter(x=df.index, y=df['upper_band'],
                        mode='lines', name='Upper', 
                        line=dict(color='blue', width=1), yaxis='y')

    trace2 = go.Scatter(x=df.index, y=df['20MA'],
                        mode='lines', name='20 MA', 
                        line=dict(color='blue', width=1, dash='dash'), yaxis='y')

    trace3 = go.Scatter(x=df.index, y=df['lower_band'],
                        mode='lines', name='Lower', 
                        line=dict(color='blue', width=1), yaxis='y')

    # Add moving averages
    trace4 = go.Scatter(x=df.index, y=df['50MA'],
                        mode='lines', name='50 MA',
                        line=dict(color='yellow', width=1), yaxis='y')

    trace5 = go.Scatter(x=df.index, y=df['200MA'],
                        mode='lines', name='200 MA',
                        line=dict(color='red', width=1), yaxis='y')

    # Add Volume and average volume
    trace6 = go.Bar(x=df.index, y=df['volume'], name='Volume', 
                    marker_color='black', yaxis='y2')

    trace7 = go.Scatter(x=df.index, y=df['avg_volume'],
                        mode='lines', name='Avg Volume', 
                        line=dict(color='red', width=1), yaxis='y2')

    # ALl the traces will be added to chart
    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
    
    # Additional layout and format of the chart
    layout = go.Layout(
        yaxis=dict(domain=[.35, 1], title='Price (USD)'),
        yaxis2=dict(domain=[0, .29], title='Volume'),

        title='{} Stock Chart'.format(ticker),

        xaxis=dict(
            title='Date',
            
            # Add range selector buttons to the chart
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=3,
                         label='3m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                         label='1y',
                         step='year',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible = False
            ),
            type='date'
        )

    )
    
    # Generate chart based on data and layout features
    fig = go.Figure(data=data, layout=layout)
    
    return fig.show()